<a href="https://colab.research.google.com/github/Atfssene/FRASA/blob/main/IndoSum_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IndoSum Dataset EDA**
**Potato Tech** team build an app that named **Frasa** for capstone project. So we will build machine learning model for Frasa to predict a summary from a sentences.

In this lab, we will do exploratory data analysis and preprocess the dataset.
The dataset is consists of news articles and manually constructed summaries [IndoSum dataset](https://drive.google.com/file/d/1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco/view) from [Kurniawan, Kemal & Louvan, Samuel (2018)](https://arxiv.org/abs/1810.05334).

Let's start!

## Preparation

In [1]:
!pip install Sastrawi

In [2]:
# Import library
import pandas as pd
import json

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize.treebank import TreebankWordDetokenizer as untoken


## Load data

In [3]:
# Download and extract the dataset
!gdown --id 1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco
!tar -xvf /content/indosum.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1OgYbPfXFAv3TbwP1Qcwt_CC9cVWSJaco
To: /content/indosum.tar.gz
96.0MB [00:00, 156MB/s]
indosum/
indosum/test.02.jsonl
indosum/README.txt
indosum/train.05.jsonl
indosum/train.04.jsonl
indosum/test.04.jsonl
indosum/train.01.jsonl
indosum/test.05.jsonl
indosum/test.03.jsonl
indosum/test.01.jsonl
indosum/dev.04.jsonl
indosum/dev.01.jsonl
indosum/dev.02.jsonl
indosum/train.02.jsonl
indosum/dev.03.jsonl
indosum/dev.05.jsonl
indosum/CHANGELOG.txt
indosum/train.03.jsonl


In [4]:
# We will use the first-fold data (train, dev, test)
files=['/content/indosum/train.01.jsonl',
       '/content/indosum/dev.01.jsonl',
       '/content/indosum/test.01.jsonl']

# url = "/content/train.json"

with open(files[0], 'r') as f, open(files[1], 'r') as ft, open(files[2], 'r') as ftest:
  data_train = f.readlines()
  data_train = list(map(json.loads, data_train))
  data_dev = ft.readlines()
  data_dev = list(map(json.loads, data_dev))
  df1 = pd.DataFrame(data_train)
  df2 = pd.DataFrame(data_dev)

  data_test = ftest.readlines()
  data_test = list(map(json.loads, data_test))

test = pd.DataFrame(data_test)
train = pd.concat([df1, df2])


## Exploratoratory

In [5]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15012 entries, 0 to 749
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     15012 non-null  object
 1   gold_labels  15012 non-null  object
 2   id           15012 non-null  object
 3   paragraphs   15012 non-null  object
 4   source       15012 non-null  object
 5   source_url   15012 non-null  object
 6   summary      15012 non-null  object
dtypes: object(7)
memory usage: 938.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     3762 non-null   object
 1   gold_labels  3762 non-null   object
 2   id           3762 non-null   object
 3   paragraphs   3762 non-null   object
 4   source       3762 non-null   object
 5   source_url   3762 non-null   object
 6   summary      3762 non-null   object
dty

Now we know the dataset consist of 7 columns and probably we will just take some of the column that related to this summarization work.

To make we sure what columns should we choose, let's take a look at some rows.

In [6]:
train.head()

,category,gold_labels,id,paragraphs,source,source_url,summary
0,tajuk utama,"[[False, True], [True, True], [False, False, F...",1501893029-lula-kamal-dokter-ryan-thamrin-saki...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dokter, R...",cnn indonesia,https://www.cnnindonesia.com/hiburan/201708041...,"[[Dokter, Lula, Kamal, yang, merupakan, selebr..."
1,teknologi,"[[False, False, False, False], [False, True, T...",1509072914-dua-smartphone-zenfone-baru-tawarka...,"[[[Selfie, ialah, salah, satu, tema, terpanas,...",dailysocial.id,https://dailysocial.id/post/dua-smartphone-zen...,"[[Asus, memperkenalkan, , ZenFone, generasi, ..."
2,hiburan,"[[True], [True], [False, False], [False], [Fal...",1510613677-songsong-visit-2020-bengkulu-perkua...,"[[[Jakarta, ,, CNN, Indonesia, -, -, Dinas, Pa...",cnn indonesia,https://www.cnnindonesia.com/gaya-hidup/201711...,"[[Dinas, Pariwisata, Provinsi, Bengkulu, kemba..."
3,tajuk utama,"[[True, True], [False, False, False], [True], ...",1502706803-icw-ada-kejanggalan-atas-tewasnya-s...,"[[[Merdeka.com, -, Indonesia, Corruption, Watc...",merdeka,https://www.merdeka.com/peristiwa/icw-merasa-a...,"[[Indonesia, Corruption, Watch, (, ICW, ), mem..."
4,tajuk utama,"[[False, True], [True, True, True], [False], [...",1503039338-pembagian-sepeda-usai-upacara-penur...,"[[[Merdeka.com, -, Presiden, Joko, Widodo, (, ...",merdeka,https://www.merdeka.com/peristiwa/usai-upacara...,"[[Jokowi, memimpin, upacara, penurunan, bender..."


From here, we can see our raw data have:

1. Tokenized text in paragraphs and summary.
2. Punctuation wasn't cleaned.
3. Lowercase and uppercase letters.
3. Column gold_labels is a boolean list of main idea sentences in paragraphs (for extractive summarization)
4. This dataset had 7 columns and 15012 rows.

For next, the columns will be used are **gold_labels, paragraphs**, and **summary**.


## Preprocessing

We will set the **paragraphs** and **summary** as paragraph instead tokenized string.

In [7]:
def untokenizer(dataframe_row, axis):
  text_list = []
  for data in dataframe_row:
    string = ""
    if axis == 0:
      for t in data:
        for text in t:
          string = string + untoken().detokenize(text) + " "
    else:
      for text in data:
          string = string + untoken().detokenize(text) + " "
    text_list.append(string)
  return text_list


In [8]:
train['paragraphs_cleaned'] = untokenizer(train['paragraphs'], 0)
train['summary_cleaned'] = untokenizer(train['summary'], 1)
test['paragraphs_cleaned'] = untokenizer(test['paragraphs'], 0)
test['summary_cleaned'] = untokenizer(test['summary'], 1)


In [9]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)

train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15012 entries, 0 to 749
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   category            15012 non-null  object
 1   gold_labels         15012 non-null  object
 2   id                  15012 non-null  object
 3   paragraphs          15012 non-null  object
 4   source              15012 non-null  object
 5   source_url          15012 non-null  object
 6   summary             15012 non-null  object
 7   paragraphs_cleaned  15012 non-null  object
 8   summary_cleaned     15012 non-null  object
dtypes: object(9)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   category            3762 non-null   object
 1   gold_labels         3762 non-null   object
 2   id                  376

In [10]:
train[['paragraphs_cleaned', 'summary_cleaned']].head(1)

,paragraphs_cleaned,summary_cleaned
0,"Jakarta, CNN Indonesia - - Dokter Ryan Thamrin, yang terkenal lewat acara Dokter Oz Indonesia, meninggal dunia pada Jumat (4 / 8) dini hari. Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan menyebut kawannya itu sudah sakit sejak setahun yang lalu. Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia. Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru, Riau untuk menjalani istirahat. "" Setahu saya dia orangnya sehat, tapi tahun lalu saya dengar dia sakit. ( Karena) sakitnya, ia langsung pulang ke Pekanbaru, jadi kami yang mau jenguk juga susah. Barangkali mau istirahat, ya betul juga, kalau di Jakarta susah isirahatnya, "" kata Lula kepada CNNIndonesia.com, Jumat (4 / 8). Lula yang mengenal Ryan sejak sebelum aktif berkarier di televisi mengaku belum sempat membesuk Ryan lantaran lokasi yang jauh. Dia juga tak tahu penyakit apa yang diderita Ryan. "" Itu saya enggak tahu, belum sempat jenguk dan enggak selamanya bisa dijenguk juga. Enggak tahu berat sekali apa bagaimana, "" tutur Ryan. Walau sudah setahun menderita sakit, Lula tak mengetahui apa penyebab pasti kematian Dr Oz Indonesia itu. Meski demikian, ia mendengar beberapa kabar yang menyebut bahwa penyebab Ryan meninggal adalah karena jatuh di kamar mandi. “ Saya tidak tahu, barangkali penyakit yang dulu sama yang sekarang berbeda, atau penyebab kematiannya beda dari penyakit sebelumnya. Kita kan enggak bisa mengambil kesimpulan, "" kata Lula. Ryan Thamrin terkenal sebagai dokter yang rutin membagikan tips dan informasi kesehatan lewat tayangan Dokter Oz Indonesia. Ryan menempuh Pendidikan Dokter pada tahun 2002 di Fakultas Kedokteran Universitas Gadjah Mada. Dia kemudian melanjutkan pendidikan Klinis Kesehatan Reproduksi dan Penyakit Menular Seksual di Mahachulalongkornrajavidyalaya University, Bangkok, Thailand pada 2004.","Dokter Lula Kamal yang merupakan selebriti sekaligus rekan kerja Ryan Thamrin menyebut kawannya itu sudah sakit sejak setahun yang lalu. Lula menuturkan, sakit itu membuat Ryan mesti vakum dari semua kegiatannya, termasuk menjadi pembawa acara Dokter Oz Indonesia. Kondisi itu membuat Ryan harus kembali ke kampung halamannya di Pekanbaru, Riau untuk menjalani istirahat."


That's better! Before continue to saving preprocessed data, let's check if it have empty value or duplicate value.

In [11]:
train.isnull().any()


category              False
gold_labels           False
id                    False
paragraphs            False
source                False
source_url            False
summary               False
paragraphs_cleaned    False
summary_cleaned       False
dtype: bool

In [12]:
test.isnull().any()

category              False
gold_labels           False
id                    False
paragraphs            False
source                False
source_url            False
summary               False
paragraphs_cleaned    False
summary_cleaned       False
dtype: bool

In [13]:
train[train['paragraphs_cleaned'].duplicated()]

,category,gold_labels,id,paragraphs,source,source_url,summary,paragraphs_cleaned,summary_cleaned


In [14]:
test[test['paragraphs_cleaned'].duplicated()]

,category,gold_labels,id,paragraphs,source,source_url,summary,paragraphs_cleaned,summary_cleaned


Great!

No empty value and no duplicate value from **15012 rows** train data and **3762 rows** test data.

That's mean we can and save both dataset to csv file.

In [19]:
csv_columns = ['gold_labels','paragraphs','summary']

train.to_csv('train.csv',
             columns=['gold_labels','paragraphs_cleaned','summary_cleaned'],
             header=csv_columns,
             index=False)
test.to_csv('test.csv',
            columns=['gold_labels','paragraphs_cleaned','summary_cleaned'],
            header=csv_columns,
            index=False)

Let's import the exported csv to make sure we had export it nicely.

In [25]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')
df_train.info()
df_train.tail(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15012 entries, 0 to 15011
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gold_labels  15012 non-null  object
 1   paragraphs   15012 non-null  object
 2   summary      15012 non-null  object
dtypes: object(3)
memory usage: 352.0+ KB


,gold_labels,paragraphs,summary
15011,"[[True], [True], [False], [True, False], [False, False], [False, False], [False], [False], [False, False], [False], [False, False], [False, False], [False], [False], [False, False], [False, False], [False], [False], [False, False]]","Jakarta, CNN Indonesia - - Ketua DPP PDIP Hendrawan Supratikno menyatakan, PDIP akan menggelar rapat internal untuk mengevaluasi dukungan terhadap Abdullah Azwar Anas menjadi calon wakil gubernur dalam Pilkada Jatim tahun 2018. Hendrawan mengatakan, rapat itu direncanakan digelar di Kantor DPP PDIP, Jakarta, Jumat (5 / 1) pukul 16.00 WIB. “ Nanti kami baru rapat pukul 16.00 WIB, ” ujar Hendrawan saat dihubungi. Hendrawan masih enggan berkomenatar soal motif di balik rencana evaluasi dukungan kepada Anas. Saat disinggung soal evaluasi karena adanya isu moral yang diduga dilakukan Anas, ia juga enggan menjawab dengan tegas. Ia beralasan, sampai saat ini belum menerima keterangan resmi dari Sekjen PDIP Hasto Kristiyanto terkait dengan isu yang mendera Anas. Namun ia mendapat informasi bahwa Hasto telah berkomunikasi dengan Anas untuk mengklarifikasi isu tersebut. “ Semuanya dipusatkan informasinya kepada Sekjen Pak Hasto. Karena kemarin Pak Hasto yang menghubungi dan mendapat tugas untuk berkomunikasi, ” ujarnya. Lebih dari itu, ia enggan, memastikan keputusan yang akan diambil PDIP atas pria yang masih menjabat Bupati Banyuwangi tersebut. “ Keputusan apa, wong baru nanti disampaikan, ” ujar Hendrawan. Sementara itu, Ketua DPP PDIP Andreas Hugo Pereira membenarkan kabar bahwa Anas mundur sebagai bakal cawagub Jatim yang diusung PDIP. Ia menyayangkan hal tersebut dilakukan Anas di tengah proses awal Pilkada. “ Sangat disayangkan pak Anas mengundurkan diri di awal proses pilkada, ” ujar Andreas saat dihubungi. Meski demikian, Andreas menduga, keputusan Anas dilakukan agar Saifullah Yusuf alias Gus Ipul dapat menang di Pilkada dengan calon lain. Keputusan Anas diduga akibat berhembusnya isu moral yang menderanya. Lebih dari itu, Andreas menyatakan, PDIP akan segera menentukan sosok yang akan menggantikan Anas sebagai cawagub Jatim. Namun, ia masih enggan menyebut Wali Kota Surabaya saat ini, Tri Rismaharini, yang akan mengganti Anas mendampingi Gus Ipul. “ Nantilah, ” ujarnya. Sementara itu, Azwar Anas menilai polemik yang mengemuka menjelang Pilkada Jatim merupakan risiko kebijakan pembangunan daerah, termasuk upaya - upaya pembunuhan karakter. "" Ada sejumlah upaya pembunuhan karakter, termasuk teror yang kerap saya diterima dan keluarga. Jadi terkait apa yang menjadi desus - desus itu, saya sudah biasa, "" ujarnya di Surabaya, Jumat (5 / 1) seperti dikutip dari Antara. Azwar Anas mengaku pembunuhan karakter bukan kali ini saja ia dapatkan. Pada tahun kedua menjabat Bupati Banyuwangi pun Anas mengaku pernah mendapatkan upaya pembunuhan karakter karena sejumlah kebijakan seperti pelarangan pasar modern dan saham untuk rakyat di sektor pertambangan. "" Bahkan, saya dilaporkan melakukan kriminalisasi kebijakan karena kebijakan-kebijakan tersebut, "" ucapnya. Azwar Anas pun mengaku dikirimi berbagai macam gambar di masa lalu untuk mencegah agar tidak mengambil kebijakan-kebijakan tertentu. "" Tapi kan saya tetap lanjutkan apa yang baik bagi orang banyak, "" ujarnya. ( kid / kid )","Ketua DPP PDIP Hendrawan Supratikno menyatakan, PDIP akan menggelar rapat internal untuk mengevaluasi dukungan terhadap Abdullah Azwar Anas menjadi calon wakil gubernur dalam Pilkada Jatim tahun 2018. Hendrawan mengatakan, rapat itu direncanakan digelar di Kantor DPP PDIP, Jakarta, Jumat (5 / 1) pukul 16.00 WIB. Hendrawan masih enggan berkomenatar soal motif di balik rencana evaluasi dukungan kepada Anas."


In [24]:
df_test.info()
df_test.tail(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gold_labels  3762 non-null   object
 1   paragraphs   3762 non-null   object
 2   summary      3762 non-null   object
dtypes: object(3)
memory usage: 88.3+ KB


,gold_labels,paragraphs,summary
3761,"[[True, False], [False, False], [False, False, False, False, False, False], [True, True], [False, False, False, False, False, False, False, False], [False, False, False], [False, False, False, False, False], [False]]","Merdeka.com - Artis senior Venna Melinda merayakan hari lahirnya yang ke - 45 pada hari Sabtu (29 / 7) kemarin. Alih - alih mengadakan pesta mewah, wanita yang kini tengah fokus di dunia politik itu memilih untuk menggelar acara syukuran bersama anak penyandang kanker di Yayasan Kanker Anak Indonesia (YKAKI) di kawasan Percetakan Negara, Jakarta Pusat, Sabtu (29 / 7). Pada hari spesialnya itu, pihak keluarga Venna ikutan hadir meramaikan, termasuk juga para buah hatinya, Athalla Naufal, Verrell Bramasta dan Vania. Yang mengejutkan lagi adalah kehadiran Natasha Wilona, yang notabene tengah santer dikabarkan dekat dengan Verrell. "" Alhamdulilah semua berjalan lancar. Keluarga aku juga kompak datang, ada Wilona juga, senang lah. Yang pasti ini bukan usia kecil lagi, aku sekarang udah punya tiga anak. Sekarang harus lebih tenang menjalani hidup. Kenapa sekarang harus lebih tenang, dulu kan lebih emosional, masih suka panikan. Sekarang berusaha lebih wise, "" ujar Venna seusai acara. Seperti diketahui, saat ini Venna tengah menyandang status janda setelah bercerai dari Ivan Fadilla tahun 2014 silam. Bintang sinetron Tersanjung 5 itu ternyata masih membuka hatinya untuk pria lain, meski saat ini lebih memilih untuk menikmati kebersamaannya bersama para buah hati lebih dulu. "" Kalau soal jodoh nggak kok. Karena aku nggak merasa sendirian. Ada Verrell, Athalla, Vania juga. Sekarang hidup aku rame jadi nggak sepi - sepi banget. Oh masih membuka (hati) kok. Cuma kan kalau di usia sekarang nggak bisa kayak orang di usia 20 ya yang harus cari-cari jodoh. Nunggu saja. Nanti kalau Tuhan sudah kasih nanti pasti tiba-tiba akan ada, "" sambungnya santai. Venna tak muluk-muluk dalam mencari pasangan hidup. Syaratnya cuma satu, siapapun calonnya nanti harus mau menerima status janda anak tiga yang saat ini disandang Venna. Lalu kira-kira sudah ada sosok yang cocok di hati? "" Nggak ada (kriteria) sih, cuma yang jelas harus bisa nerima aku dengan anak - anakku. Cari orang yang model begitu zaman sekarang ya nggak mudah. Apalagi dengan kedudukan aku, orang kadang ada yang ciut. Padahal biasa saja. Aduh, belum, belum ada yang sesuai, "" pungkasnya. ( kpl / abs / gtr )","Artis senior Venna Melinda merayakan hari lahirnya yang ke - 45, sabtu kemarin. Vena memilih untuk menggelar acara syukuran bersama anak penyandang kanker di YKAKI. Seperti diketahui, saat ini Venna tengah menyandang status janda setelah bercerai dari Ivan Fadilla tahun 2014 silam. Bintang sinetron ini ternyata masih membuka hatinya untuk pria lain."


Wonderful! Now we can build our text summarization model.

For this project, BERT-Large-CNN will be used for the pretrained model and we will train it with this dataset.